In [1]:
import torch
from schnetpack.datasets import QM9
import schnetpack as spk
import os
from my_config import config_args
from MyModel.HGCN import RegModel
import optimizers
import numpy as np
import logging
import time
from tqdm import tqdm

# 获得训练数据

In [2]:
qm9data = QM9('./data/qm9.db', download=True,load_only=[QM9.U0])
qm9split = './data/qm9split'
print(len(qm9data))

133885


In [3]:
train, val, test = spk.train_test_split(
        data=qm9data,
        num_train=20000,
        num_val=10000,
        split_file=os.path.join(qm9split, "split20000.npz"),
    )
print(len(train),len(test),len(val))

20000 103885 10000


In [4]:
train_loader = spk.AtomsLoader(train, batch_size=400, shuffle=False)
val_loader = spk.AtomsLoader(val, batch_size=10)

In [5]:
from manifolds.hyperboloid import Hyperboloid
manifold = Hyperboloid()
size = (1,3,2)
input = torch.ones(size)
c = 1
input = torch.concat([torch.zeros((1,3,1)),input],dim=2)
input = manifold.proj_tan0(input,c)
print(input)

tensor([[[0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.]]])


In [6]:
input = manifold.expmap0(input,c)

In [7]:
import sys
import json
class obj(object):
    def __init__(self, dict_):
        self.__dict__.update(dict_)
args = json.loads(json.dumps(config_args), object_hook=obj)

model = RegModel(args)
optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr,
                                                    weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=args.lr_reduce_freq,
    gamma=float(args.gamma)
)
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
logging.info(f"Total number of parameters: {tot_params}")
device = torch.device('cuda')
# Train model
t_total = time.time()
model = model.to(device)
loss_fun = torch.nn.MSELoss()
for epoch in range(args.epochs):
    loss_sum,n,t = 0,0,0.0
    counter = 0
    for input in tqdm(train_loader):
        counter+=1
        # if counter>10:
        #     sys.exit(0)
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        y_hat = model(input)
        # print('y_hat',y_hat[:2])
        # print('input[QM9.U0]',input[QM9.U0][:2])
        loss = loss_fun(y_hat,input[QM9.U0])


        # print(loss)
        loss.backward()
        loss_sum += loss
        n += 1
        if args.grad_clip is not None:
            max_norm = float(args.grad_clip)
            all_params = list(model.parameters())
            for param in all_params:
                torch.nn.utils.clip_grad_norm_(param, max_norm)
        optimizer.step()
    lr_scheduler.step()
    if (epoch + 1) % args.log_freq == 0:
        str = " ".join(['Epoch: {:04d}'.format(epoch + 1),
                               'lr: {}'.format(lr_scheduler.get_last_lr()[0]),
                               'loss: {:.4f}'.format(loss_sum/n),
                               'time: {:.4f}s'.format(time.time() - t)
                               ])
        print(str)
        logging.info(str)
        curvatures = list(model.get_submodule('curvatures'))
        print(curvatures)



    # if (epoch + 1) % args.eval_freq == 0:
    #     model.eval()
    #     embeddings = model.encode(data['features'], data['adj_train_norm'])
    #     val_metrics = model.compute_metrics(embeddings, data, 'val')
    #     if (epoch + 1) % args.log_freq == 0:
    #         logging.info(" ".join(['Epoch: {:04d}'.format(epoch + 1), format_metrics(val_metrics, 'val')]))
    #     if model.has_improved(best_val_metrics, val_metrics):
    #         best_test_metrics = model.compute_metrics(embeddings, data, 'test')
    #         best_emb = embeddings.cpu()
    #         if args.save:
    #             np.save(os.path.join(save_dir, 'embeddings.npy'), best_emb.detach().numpy())
    #         best_val_metrics = val_metrics
    #         counter = 0
    #     else:
    #         counter += 1
    #         if counter == args.patience and epoch > args.min_epochs:
    #             logging.info("Early stopping")
    #             break

    """
    Epoch: 0027 lr: 0.01 loss: 1459.0519 time: 0.0866s

    """

100%|██████████| 50/50 [00:28<00:00,  1.78it/s]


Epoch: 0001 lr: 0.01 loss: 51961592.0000 time: 0.0675s
[Parameter containing:
tensor([1.0640], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9447], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9659], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0014], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0280], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0337], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0220], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8107], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9005], device='cuda:0', requires_grad=True)]


100%|██████████| 50/50 [00:26<00:00,  1.90it/s]


Epoch: 0002 lr: 0.01 loss: 5395310.5000 time: 0.0674s
[Parameter containing:
tensor([1.0635], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9426], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9663], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0762], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9837], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0745], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9929], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8082], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.9121], device='cuda:0', requires_grad=True)]


100%|██████████| 50/50 [00:26<00:00,  1.89it/s]


...

Epoch: 0067 lr: 0.00125 loss: 3.7545 time: 0.0811s
[Parameter containing:
tensor([1.4120], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8183], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6921], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5693], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.3667], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0216], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8011], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.7809], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8212], device='cuda:0', requires_grad=True)]


100%|██████████| 50/50 [00:26<00:00,  1.90it/s]


Epoch: 0068 lr: 0.00125 loss: 3.7267 time: 0.0836s
[Parameter containing:
tensor([1.4122], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8184], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6919], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5692], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.3665], device='cuda:0', requires_grad=True), Parameter containing:
tensor([1.0224], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8011], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.7809], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.8212], device='cuda:0', requires_grad=True)]


 12%|█▏        | 6/50 [00:03<00:26,  1.69it/s]


KeyboardInterrupt: 

In [ ]:
# from manifolds.hyperboloid import Hyperboloid
# import torch
# size = (2,5,10)
# a = 2*torch.ones(size)
# a = torch.concat([torch.zeros((2,5,1)),a],dim=2)
# print(a)
# hyper = Hyperboloid()
# c = 1
# a_H = hyper.expmap0(a,c)  #
# print(a_H)
# a_H_p = hyper.proj(a_H,c)  #对双曲空间的向量没有影响
# print(a_H_p)

# a_tan = hyper.logmap0(a_H,c)
# print(a_tan)
#
# a_tan_p = hyper.proj_tan0(a_tan,c)  #  对已经在o点切空间的向量没有影响
# print(a_tan_p)

# a_tan_a = hyper.logmap(a_H,a_H,c)  # 向量在其切空间是0
# print(a_tan_a)



